In [4]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
pages = ["http://sg.startupjobs.asia/sg/job/search?w=jobs&q=data+scientist+OR+data+analyst+OR+business+analyst+OR+business+intelligence&l=Anywhere&t=any&job_page=" + str(i) for i in range(1, 12)]

In [8]:
job_links = []

for url in pages:

    r  = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "lxml")

    container = soup.find_all('div', attrs={'id': 'yw0'})
    for text in container:
        job_names = text.find_all('span', attrs={'class': 'JobRole'})
        for name in job_names: 
            for link in name.find_all('a'):
                job_link = link.get('href')
                job_links.append("http://sg.startupjobs.asia" + str(job_link))

In [17]:
len(job_links)

324

In [41]:
def extract_all_information(job_links):
    
    jobTitles = []
    jobTypes = []
    countries = []
    companyDescs = []
    companyNames = []
    resps = []
    reqs = []
    salaries = []
    applicants = []
    posting_dates = []
    total_views = []
    uniq_views = []
    
    for link in job_links:
        r  = requests.get(link)
        data = r.text
        soup = BeautifulSoup(data)
        
        body = soup.find('div', attrs={'class': 'jobBody'})
        title = soup.find('h1', attrs={'class': 'joblistingpagetitle'})
        company = soup.find('div', attrs={'class': 'type companynametitle'})
        country = soup.find('div', attrs={'class': 'type companycountry'})
        jobtype = soup.find('div', attrs={'class': 'clear type m_title companycountry'})
        add_info = soup.find_all('div', attrs={'class': 'clear'})
        
        # Extract company name
        try:
            company_name = [x.strip() for x in re.findall('<div class="type companynametitle" style="float:none;font-weight:bold;margin-bottom: 18px;">(.*)</div>', str(company), re.MULTILINE | re.DOTALL)]
            companyNames.append(company_name)
        except AttributeError:
            companyNames.append("")
            
        # Extract job title
        try:
            job_title = re.search('<h1 class="joblistingpagetitle">(.+?)</h1>', str(title)).group(1)
            jobTitles.append(job_title)
        except AttributeError:
            jobTitles.append("")
            
        # Extract job type
        try:
            job_type = [x.strip() for x in re.findall('<div class="clear type m_title companycountry">(.*)</div>', str(jobtype), re.MULTILINE | re.DOTALL)]
            jobTypes.append(job_type)
        except AttributeError:
            jobTypes.append("")
            
        # Extract country
        try:
            job_country = [x.strip() for x in re.findall('<div class="type companycountry" style="float:left;margin-top:-10px;margin-bottom:-2px;">(.+?)</div>', str(country), re.MULTILINE | re.DOTALL)]
            countries.append(job_country)
        except AttributeError:
            countries.append("")
            
        # Extract company description
        try:
            desc = [x.strip() for x in re.findall('<div class="clear jobBody">(.+?)Responsibilities', str(body), re.MULTILINE | re.DOTALL)]
            companyDescs.append(desc)
        except AttributeError:
            companyDescs.append("")
            
        # Extract responsibilites info
        try:
            resp = [x.strip() for x in re.findall('Responsibilities(.+?)Requirements', str(body), re.MULTILINE | re.DOTALL)]
            resps.append(resp)
        except AttributeError:
            resps.append("")

        # Extract requirements info
        try:
            req = [x.strip() for x in re.findall('Requirements(.+?)</div>', str(body), re.MULTILINE | re.DOTALL)]
            reqs.append(req)
        except AttributeError:
            reqs.append("")

        # Extract salary info
        try:
            salary = [x.strip() for x in re.findall('<h4 class="joblistingpagetitle"> Salary </h4>(.+?)</div>', str(body), re.MULTILINE | re.DOTALL)]
            salaries.append(salary)
        except AttributeError:
            salaries.append("")
            
        # Extract total applicants
        try:      
            tot_app = [x.strip() for x in re.findall('<span class="label">(.+?)</span>', str(add_info), re.MULTILINE | re.DOTALL)]
            applicants.append(tot_app)
        except AttributeError:
            applicants.append("")
        
        # Extract job posting date
        try:
            date = [x.strip() for x in re.findall('<span class="label label-info">(.+?)</span>', str(add_info), re.MULTILINE | re.DOTALL)]
            posting_dates.append(date)
        except AttributeError:
            posting_dates.append("")
            
        # Extract total views
        try:
            view_count = [x.strip() for x in re.findall('<span class="label label-success">(.+?)</span>', str(add_info), re.MULTILINE | re.DOTALL)]
            total_views.append(view_count)
        except AttributeError:
            total_views.append("")
            
        # Extract unique views
        try:
            uniq_view_count = [x.strip() for x in re.findall('<span class="label label-warning">(.+?)</span>', str(add_info), re.MULTILINE | re.DOTALL)]
            uniq_views.append(uniq_view_count)
        except AttributeError:
            uniq_views.append("")
            
    results = pd.DataFrame({ "link": job_links,
                             "company name": companyNames,
                             "job title": jobTitles,
                             "job type": jobTypes,
                             "country": countries,
                             "company desc": companyDescs,
                             "responsibilities": resps,
                             "requirements": reqs,
                             "salary": salaries,
                             "total applicants": applicants,
                             "job posting date": posting_dates,
                             "total views": total_views,
                             "unique views": uniq_views})

    return results

In [43]:
results = extract_all_information(job_links)

In [46]:
results

,company desc,company name,country,job posting date,job title,job type,link,requirements,responsibilities,salary,total applicants,total views,unique views
0,[<p>Tuple (www.tuple-tech.com) is a VC backed ...,[Tuple Technologies Pte Ltd],[Country : Singapore],[Job posted 1 month ago],Lead Data Scientist,"[<ul>\n<li class=""Full-time"">Full-time</li></ul>]",http://sg.startupjobs.asia/sg/job/35451-lead-d...,[</h3>\n<p> </p><p>- Strong background in Mach...,[</h3>\n<p> </p><p><strong>NOTE: This position...,[4000 - 6000 SGD],[Total applicants :3],[Total Views : 360],[Unique Views : 357]
1,[<p>LotusFlare’s mission is to make mobile int...,[LotusFlare Inc],[Country : Singapore],[Job posted 29 days ago],Data Analyst-Data Scientist,"[<ul>\n<li class=""Full-time"">Full-time</li></ul>]",http://sg.startupjobs.asia/sg/job/35619-data-a...,[</h3>\n<ul><li>You have a Ph.D. or Master’s D...,"[</h3>\n<ul><li>You will advocate, evangelize ...",[],[Total applicants :34],[Total Views : 453],[Unique Views : 449]
2,[<p><strong><u>[MoolahSense] Help SMEs expand ...,[MoolahSense Private Limited],[Country : Singapore],[Job posted 6 months ago],Data Scientist,"[<ul>\n<li class=""Full-time"">Full-time</li></ul>]",http://sg.startupjobs.asia/sg/job/31881-data-s...,"[</h3>\n<ul><li>Knowledge of tools like R, SQL...",[</h3>\n<ul><li>Introduce new innovative model...,[],[Total applicants :100],[Total Views : 1435],[Unique Views : 1432]
3,[<p>Change the way people eat everyday.</p><p>...,[dahmakan],[Country : Anywhere],[Job posted 5 days ago],Data analyst,"[<ul>\n<li class=""Full-time"">Full-time</li></ul>]",http://sg.startupjobs.asia/sg/job/36145-data-a...,[</h3>\n<ol><li><strong>Technical skills</stro...,[</h3>\n<ul><li>Be the left brain behind every...,[4000 - 9000 MYR],[Total applicants :5],[Total Views : 158],[Unique Views : 155]
4,"[<p>Founded in 2009, SmartRetail is an innovat...",[SmartRetail],[Country : Singapore],[Job posted 2 months ago],Business Development Associate,"[<ul>\n<li class=""Full-time"">Full-time</li></ul>]",http://sg.startupjobs.asia/sg/job/35189-busine...,[</h3>\n<ul><li>Entrepreneurial and hungry. Ab...,[</h3>\n<ol><li>Develop sales activity for all...,[2500 - 3000 SGD],[Total applicants :20],[Total Views : 346],[Unique Views : 343]
5,[<p>SmartBuild Asia aims to be disruptive! It ...,[SmartBuild Asia],[Country : Singapore],[Job posted 4 months ago],Business Development &amp; Sales - Managers &a...,"[<ul>\n<li class=""Full-time"">Full-time</li><li...",http://sg.startupjobs.asia/sg/job/33057-busine...,[</h3>\n<p>Importantly we are recruiting a tea...,"[</h3>\n<p>SmartBuild Asia has a lot of data, ...",[2500 - 6000 SGD],[Total applicants :46],[Total Views : 736],[Unique Views : 733]
6,"[<p>At Healint, our mission is to bring wild i...",[Healint Pte Ltd],[Country : Singapore],[Job posted 2 months ago],Junior Data Scientist,"[<ul>\n<li class=""Full-time"">Full-time</li></ul>]",http://sg.startupjobs.asia/sg/job/35133-junior...,[</h3>\n<p><!--[if gte mso 9]><xml>\r\n<o:Docu...,[</h3>\n<p><!--[if gte mso 9]><xml>\r\n<o:Docu...,[],[Total applicants :99],[Total Views : 546],[Unique Views : 543]
7,[<p>Looking to immerse yourself right at the h...,[The Oddle Company],[Country : Singapore],[Job posted 7 days ago],Product Analyst,"[<ul>\n<li class=""Full-time"">Full-time</li></ul>]",http://sg.startupjobs.asia/sg/job/36079-produc...,[</h3>\n<p><strong>Requirements:</strong></p><...,"[</h3>\n<p>By joining our Product team, you wi...",[],[Total applicants :4],[Total Views : 163],[Unique Views : 161]
8,[<p><strong>About the Company</strong> <br/>Ci...,[Circles.Life],[Country : Singapore],[Job posted 2 months ago],Data Analyst,"[<ul>\n<li class=""Full-time"">Full-time</li></ul>]",http://sg.startupjobs.asia/sg/job/34926-data-a...,[</h3>\n<p><strong>Requirements</strong></p><u...,[</h3>\n<p>We have an opening for a <strong>Da...,[],[Total applicants :40],[Total Views : 560],[Unique Views : 557]
9,[<p><!--[if gte mso 9]><xml>\r\n<o:OfficeDocum...,[ThunderQ